In [72]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from src.augmentations import CustomAugmentations


In [73]:
height,width=32,32
batch_size=64

In [74]:
# import Sequential from keras
from tensorflow.keras.models import Sequential


dnn_model = Sequential()
imported_model= tf.keras.applications.EfficientNetV2L (include_top=False,
input_shape=(height,width,3),
pooling='max',classes=10,
weights='imagenet')

from tensorflow.keras.layers import Dense, Flatten
dnn_model.add(imported_model)
dnn_model.add(Flatten())
dnn_model.add(Dense(2048, activation='relu'))
dnn_model.add(Dense(1024, activation='relu'))
dnn_model.add(Dense(10, activation='softmax'))

dnn_model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten_2 (Flatten)         (None, 2048)              0         
                                                                 
 dense_6 (Dense)             (None, 2048)              4196352   
                                                                 
 dense_7 (Dense)             (None, 1024)              2098176   
                                                                 
 dense_8 (Dense)             (None, 10)                10250     
                                                                 
Total params: 29,892,490
Trainable params: 29,839,370
Non-trainable params: 53,120
_________________________________________________________________


In [75]:
from tensorflow.keras.optimizers import Adam
dnn_model.compile(optimizer=Adam(lr=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

c:\Users\adamm\miniconda3\envs\py310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [76]:
train_set = tf.keras.preprocessing.image_dataset_from_directory(
    '../data/CINIC10/train',
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(height, width),
    shuffle=True,
    seed=123,
    validation_split=0.05,
    subset='training'
)

val_set = tf.keras.preprocessing.image_dataset_from_directory(
    '../data/CINIC10/valid',
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(height, width),
    shuffle=True,
    seed=123,
    validation_split=0.05,
    subset='validation'
)

test_set = tf.keras.preprocessing.image_dataset_from_directory(
    '../data/CINIC10/test',
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(height, width),
    shuffle=False,
)


In [ ]:
from copy import deepcopy
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
from tqdm.notebook import tqdm

histories = list()
accuracies = list()
class_accuracies = list()
conf_mats = list()

for i in tqdm(range(5)):
    print(f'Fitting model {i}')
    history = dnn_model.fit(train_set, validation_data=val_set, epochs=25)

    print(f'Predicting model {i}')
    preds = dnn_model.predict(test_set, verbose=2)
    preds = preds.argmax(axis=1)
    classes = test_set.class_names
    test_labels = list()
    for images, labels in test_set:
        class_labels = [int(label) for label in labels]
        test_labels.extend(class_labels)
    test_labels = np.array(test_labels)

    conf_mat = confusion_matrix(test_labels, preds)
    accuracy = accuracy_score(test_labels, preds)
    class_accuracy = conf_mat.diagonal()/conf_mat.sum(axis=1)

    print(f'Saving model {i}')
    dnn_model.save(f'../models/EfficientNetV2L_no_aug_{i}.h5')
    histories.append(deepcopy(history.history))
    accuracies.append(accuracy)
    class_accuracies.append(class_accuracy)
    conf_mats.append(conf_mat)

    
# save histories
import pickle
with open('../models/HISTORY_EfficientNetV2L_no_aug.pkl', 'wb') as f:
    pickle.dump(histories, f)

# save accuracies
with open('../models/ACCURACY_EfficientNetV2L_no_aug.pkl', 'wb') as f:
    pickle.dump(accuracies, f)

# save class accuracies
with open('../models/CLASS_ACCURACY_EfficientNetV2L_no_aug.pkl', 'wb') as f:
    pickle.dump(class_accuracies, f)

# save confusion matrices
with open('../models/CONF_MAT_EfficientNetV2L_no_aug.pkl', 'wb') as f:
    pickle.dump(conf_mats, f)




ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html